# Trying to fix the 47 Tuc orbital pulsars fitting that broke somehow

At some point in the past month the orbital (binary) pulsar likelihoods seem to have broken, and are always failing (i.e. returning -inf).
This notebook is to try and figure out why.

In [1]:
import fitter
from fitter.probabilities.pulsars import *

import numpy
import matplotlib.pyplot as plt
import astropy.units as u
import scipy.interpolate as interp

%matplotlib widget


In [2]:
obs = fitter.Observations('47_Tuc')
# theta = obs.initials.values()
theta = [6.1, 1.06, 8.1, 1.23, 0.7, 0.45, 0.1, 0.45, 0.5, 1.3, 2.5, 0.5, 4.45]
# theta = [6.347, 0.967, 7.468, 1.282, 0.841, 0.454, 0.006, 0.268, 0.589, 0.553, 2.589, 0.204, 4.439]

model = fitter.Model(theta, obs)

u.set_enabled_equivalencies(fitter.util.angular_width(model.d))
*_, μ, crd = obs.valid_likelihoods[-3]

/home/dev/Documents/SMU/Research/dev/GCfit/fitter/probabilities/pulsars.py:254: RuntimeWarning: invalid value encountered in log10
  Pdot_int = np.log10(Pdot_int)


In [3]:
def likelihood_pulsar_orbital(model, pulsars, cluster_μ, coords, *, mass_bin=None):
    
    # ----------------------------------------------------------------------
    # Get pulsar mass bins
    # ----------------------------------------------------------------------

    if mass_bin is None:
        if 'm' in pulsars.mdata:
            mass_bin = np.where(model.mj == pulsars.mdata['m'] * u.Msun)[0][0]
        else:
            logging.debug("No mass bin provided for pulsars, using -1")
            mass_bin = -1

    # ----------------------------------------------------------------------
    # Iterate over all pulsars
    # ----------------------------------------------------------------------

    N = pulsars['r'].size
    probs = np.zeros(N)

    fig, axarr = plt.subplots(N, 3, figsize=(13, 25))

    for i in range(N):

        axrow = axarr[i]

        # ------------------------------------------------------------------
        # Get this pulsars necessary data
        # ------------------------------------------------------------------

        id_ = pulsars['id'][i].value.decode()
        axrow[0].set_title(f'Pulsar "{id_}"')

        print(i, id_)
        
        R = pulsars['r'][i].to(u.pc)

        Pb = pulsars['Pb'][i].to('s')

        Pbdot_meas = pulsars['Pbdot_meas'][i]
        ΔPbdot_meas = pulsars['ΔPbdot_meas'][i]
        
        axrow[1].set_title(f'$\dot{{P_b}}=${Pbdot_meas.value} ($\pm$ {ΔPbdot_meas.value})')

        # ------------------------------------------------------------------
        # Compute the cluster component distribution, from the model
        # ------------------------------------------------------------------

        PdotP_domain, PdotP_c_prob = cluster_component(model, R, mass_bin)
        Pdot_domain = (Pb * PdotP_domain).decompose()

        axrow[0].plot(Pdot_domain, PdotP_c_prob, 'r-')

        # linear to avoid effects around asymptote
        Pdot_c_spl = interp.UnivariateSpline(
            Pdot_domain, PdotP_c_prob, k=1, s=0, ext=1
        )

        # ------------------------------------------------------------------
        # Compute gaussian measurement error distribution
        # ------------------------------------------------------------------

        err = fitter.util.gaussian(x=Pdot_domain, sigma=ΔPbdot_meas, mu=0)

        err_spl = interp.UnivariateSpline(Pdot_domain, err, k=1, s=0, ext=1)

        axrow[1].plot(Pdot_domain, err, 'g-')

        # ------------------------------------------------------------------
        # Set up the equally-spaced linear convolution domain
        # ------------------------------------------------------------------

        # mirrored/starting at zero so very small gaussians become the δ-func
        lin_domain = np.linspace(0., 1e-11, 5_000 // 2)
        lin_domain = np.concatenate((np.flip(-lin_domain[1:]), lin_domain))

        axrow[0].plot(lin_domain, Pdot_c_spl(lin_domain), 'r--')
        axrow[1].plot(lin_domain, err_spl(lin_domain), 'g--')
        
        # Testing the err without this small domain
        err2 = fitter.util.gaussian(x=lin_domain, sigma=ΔPbdot_meas, mu=0)

        axrow[1].plot(lin_domain, err2, 'm-.')

        
        # ------------------------------------------------------------------
        # Convolve the different distributions
        # ------------------------------------------------------------------

        # conv = np.convolve(err, PdotP_c_prob, 'same')
        conv = np.convolve(err_spl(lin_domain), Pdot_c_spl(lin_domain), 'same')

        # Normalize
        conv /= interp.UnivariateSpline(
            lin_domain, conv, k=1, s=0, ext=1
        ).integral(-np.inf, np.inf)
        
        conv2 = np.convolve(err2, Pdot_c_spl(lin_domain), 'same')

        # Normalize
        conv2 /= interp.UnivariateSpline(
            lin_domain, conv2, k=1, s=0, ext=1
        ).integral(-np.inf, np.inf)

        # ------------------------------------------------------------------
        # Compute the Shklovskii (proper motion) effect component
        # ------------------------------------------------------------------

        cluster_μ <<= u.Unit("mas/yr")

        PdotP_pm = shklovskii_component(cluster_μ, model.d)

        # ------------------------------------------------------------------
        # Compute the galactic potential component
        # ------------------------------------------------------------------

        PdotP_gal = galactic_component(*(coords * u.deg), D=model.d)

        # ------------------------------------------------------------------
        # Interpolate the likelihood value from the overall distribution
        # ------------------------------------------------------------------

        prob_dist = interp.interp1d(
            (lin_domain / Pb) + PdotP_pm + PdotP_gal, conv,
            assume_sorted=True, bounds_error=False, fill_value=0.0
        )

        probs[i] = prob_dist(Pbdot_meas / Pb)
        
        print(probs[i])
        
        axrow[2].plot((lin_domain / Pb) + PdotP_pm + PdotP_gal, conv, 'b-')
        axrow[2].plot((lin_domain / Pb) + PdotP_pm + PdotP_gal, conv2, 'm--')
        axrow[2].axvline((Pbdot_meas / Pb).value)

    # ----------------------------------------------------------------------
    # Multiply all the probabilities and return the total log probability.
    # ----------------------------------------------------------------------

    fig.tight_layout()
    
    return np.sum(np.log(probs))


In [4]:
likelihood_pulsar_orbital(model, obs['pulsar/orbital'], μ, crd)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 J0024-7204E
8.770469674185307e-09
1 J0024-7204H
192026382636.10727
2 J0024-7204I
35486348552.73593
3 J0024-7204Q
254656713068.67194
4 J0024-7204R
802727329239.4203
5 J0024-7204S
651.4304668236284
6 J0024-7204T
91641071623.92317
7 J0024-7204U
275415.1221625323
8 J0024-7204X
0.0
9 J0024-7204Y
496738353916.99786


<ipython-input-3-c689e00e69c9>:142: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(probs))


-inf

In [6]:
plt.close('all')

So a couple things, it looks like. Firstly, the fix I came up with before to requiring the orbital likelihood to use the lin_domain, just like the spin likelihood, seems like the right
choice, the new cuts on the cluster_component (and the lack of zeroed tails after it) was justified by me claiming it would always be used in a spline with constant 0 outside the boundaries. This wasn't the case before in the orbital, cause I guess I forgot about that, so that was a needed fix. Of course this means the lin_domain has to be set up correctly as
well, you can't just copy from spin cause the order of magnitudes are way different. That was also a mistake.

-> I'm wondering if it might be smart to start enforcing this, a la returning a spline instead of the domain and probability?

Secondly, this Pdot_domain was being used to create the gaussian error curves. This was a problem, considering the new cutoff, cause the full gaussian tails weren't present, or allowed
to go to zero, and so the splines, with explicitly zeroed boundaries, were creating weird cutoff gaussians, which was what was leading to the strange triangle final convolution curves.
I think this may also still be an issue in the spin likelihoods, although I don't know how much problems it's causing considering it's only really noticeable when the gaussian curve is
large compared to the others, and in the spin it's always dominated by either the cluster or the intrinsic (I believe)

I'm not super happy with how this has evolved to now pretty much requiring a dense, arbitrary lin_domain for every part. It feels like this is going back to what wew started with, and
I'm not really sure what changes I've made over the past months have actually stuck around as intended.

I think the whole organization of the pulsars stuff might require another look.

# Let's check out the spin one as well, cause that's also not working great

In [5]:
def likelihood_pulsar_spin(model, pulsars, Pdot_kde, cluster_μ, coords, *, mass_bin=None):

    # ----------------------------------------------------------------------
    # Get the pulsar P-Pdot_int kde
    # ----------------------------------------------------------------------

    if Pdot_kde is None:
        Pdot_kde = field_Pdot_KDE()

    Pdot_min, Pdot_max = Pdot_kde.dataset[1].min(), Pdot_kde.dataset[1].max()

    # ----------------------------------------------------------------------
    # Get pulsar mass bins
    # ----------------------------------------------------------------------

    if mass_bin is None:
        if 'm' in pulsars.mdata:
            mass_bin = np.where(model.mj == pulsars.mdata['m'] * u.Msun)[0][0]
        else:
            logging.debug("No mass bin provided for pulsars, using -1")
            mass_bin = -1

    # ----------------------------------------------------------------------
    # Iterate over all pulsars
    # ----------------------------------------------------------------------

    N = pulsars['r'].size
    probs = np.zeros(N)

    fig, axarr = plt.subplots(N, 4, figsize=(13, 35), sharex='row')
        
    for i in range(N):

        axrow = axarr[i]

        # ------------------------------------------------------------------
        # Get this pulsars necessary data
        # ------------------------------------------------------------------

        id_ = pulsars['id'][i].value.decode()
        axrow[0].set_title(f'Pulsar "{id_}"')
        
        print(i, id_)
        
        R = pulsars['r'][i].to(u.pc)

        P = pulsars['P'][i].to('s')

        Pdot_meas = pulsars['Pdot_meas'][i]
        ΔPdot_meas = np.abs(pulsars['ΔPdot_meas'][i])
        
        axrow[1].set_title(f'$\dot{{P}}=${Pdot_meas.value} ($\pm$ {ΔPdot_meas.value})')

        # ------------------------------------------------------------------
        # Compute the cluster component distribution, from the model
        # ------------------------------------------------------------------

        PdotP_domain, PdotP_c_prob = cluster_component(model, R, mass_bin)
        Pdot_domain = (P * PdotP_domain).decompose()
        
        # linear to avoid effects around asymptote
        Pdot_c_spl = interp.UnivariateSpline(
            Pdot_domain, PdotP_c_prob, k=1, s=0, ext=1
        )

        # ------------------------------------------------------------------
        # Compute gaussian measurement error distribution
        # ------------------------------------------------------------------

        # TODO if width << Pint width, maybe don't bother with first conv.

        err = fitter.util.gaussian(x=Pdot_domain, sigma=ΔPdot_meas, mu=0)
        
        err_spl = interp.UnivariateSpline(Pdot_domain, err, k=1, s=0, ext=1)

        # ------------------------------------------------------------------
        # Create a slice of the P-Pdot space, along this pulsars P
        # ------------------------------------------------------------------

        lg_P = np.log10(P / P.unit)

        P_grid, Pdot_int_domain = np.mgrid[lg_P:lg_P:1j, Pdot_min:Pdot_max:200j]

        P_grid, Pdot_int_domain = P_grid.ravel(), Pdot_int_domain.ravel()

        # ------------------------------------------------------------------
        # Compute the Pdot_int distribution from the KDE
        # ------------------------------------------------------------------

        Pdot_int_prob = Pdot_kde(np.vstack([P_grid, Pdot_int_domain]))

        Pdot_int_spl = interp.UnivariateSpline(
            Pdot_int_domain, Pdot_int_prob, k=1, s=0, ext=1
        )

        Pdot_int_prob = fitter.util.RV_transform(
            domain=10**Pdot_int_domain, f_X=Pdot_int_spl,
            h=np.log10, h_prime=lambda y: (1 / (np.log(10) * y))
        )

        Pdot_int_spl = interp.UnivariateSpline(
            10**Pdot_int_domain, Pdot_int_prob, k=1, s=0, ext=1
        )

        # ------------------------------------------------------------------
        # Set up the equally-spaced linear convolution domain
        # ------------------------------------------------------------------

        # TODO both 5000 and 1e-18 need to be computed dynamically
        #   5000 to be enough steps to sample the gaussian and int peaks
        #   1e-18 to be far enough for the int distribution to go to zero
        #   Both balanced so as to use way too much memory uneccessarily
        #   Must be symmetric, to avoid bound effects

        # mirrored/starting at zero so very small gaussians become the δ-func
        lin_domain = np.linspace(0., .5e-18, 5_000 // 2)
        lin_domain = np.concatenate((np.flip(-lin_domain[1:]), lin_domain))
        
        # Testing the err without this small domain
        err2 = fitter.util.gaussian(x=lin_domain, sigma=ΔPdot_meas, mu=0)

        # ------------------------------------------------------------------
        # Convolve the different distributions
        # ------------------------------------------------------------------

        conv1 = np.convolve(err_spl(lin_domain), Pdot_c_spl(lin_domain), 'same')

        conv2 = np.convolve(conv1, Pdot_int_spl(lin_domain), 'same')

        # Normalize
        conv2 /= interp.UnivariateSpline(
            lin_domain, conv2, k=1, s=0, ext=1
        ).integral(-np.inf, np.inf)

        # ------------------------------------------------------------------
        # Compute the Shklovskii (proper motion) effect component
        # ------------------------------------------------------------------

        cluster_μ <<= u.Unit("mas/yr")

        PdotP_pm = shklovskii_component(cluster_μ, model.d)

        # ------------------------------------------------------------------
        # Compute the galactic potential component
        # ------------------------------------------------------------------

        PdotP_gal = galactic_component(*(coords * u.deg), D=model.d)

        # ------------------------------------------------------------------
        # Interpolate the likelihood value from the overall distribution
        # ------------------------------------------------------------------

        xtot = (lin_domain / P) + PdotP_pm + PdotP_gal
        
        prob_dist = interp.interp1d(
            xtot, conv2,
            assume_sorted=True, bounds_error=False, fill_value=0.0
        )

        probs[i] = prob_dist((Pdot_meas / P).decompose())
        
        print(probs[i])

#         xtot = lin_domain
        
        
#         axrow[0].plot(Pdot_domain, PdotP_c_prob, 'r-')
#         axrow[0].plot(lin_domain, Pdot_c_spl(lin_domain), 'r--')
        axrow[0].plot(xtot, Pdot_c_spl(lin_domain), 'r--')

#         axrow[1].plot(Pdot_domain, err, 'g-')
#         axrow[1].plot(lin_domain, err_spl(lin_domain), 'g--')
#         axrow[1].plot(lin_domain, err2, 'm-.')
        axrow[1].plot(xtot, err2, 'm-.')

        axrow[2].plot(xtot, Pdot_int_spl(lin_domain), 'c-')
#         axrow[2].plot(lin_domain, Pdot_int_spl(lin_domain), 'c-')

        axrow[3].plot(xtot, conv2, 'b-')
        axrow[3].axvline((Pdot_meas / P).value)

    # ----------------------------------------------------------------------
    # Multiply all the probabilities and return the total log probability.
    # ----------------------------------------------------------------------

    logprobs = np.log(probs)

    # Should never occur anymore, but leave it here for now just in case
    logprobs[np.isnan(logprobs)] = np.NINF

    fig.tight_layout()
    
    return np.sum(logprobs)



In [6]:
likelihood_pulsar_spin(model, obs['pulsar/spin'], None, μ, crd)

/home/dev/Documents/SMU/Research/dev/GCfit/fitter/probabilities/pulsars.py:254: RuntimeWarning: invalid value encountered in log10
  Pdot_int = np.log10(Pdot_int)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 J0024-7204ab
4.4894990038233907e+18
1 J0024-7204c
0.0
2 J0024-7204D
4.767643832212169e+18
3 J0024-7204E
1.835535588708238e+18
4 J0024-7204F
7.118899792902426e+18
5 J0024-7204G
3.8948322899374423e+18
6 J0024-7204H
8.661621573310227e+18
7 J0024-7204I
4.215350433672365e+18
8 J0024-7204J
1.1855351316523012e+19
9 J0024-7204L
0.0
10 J0024-7204M
0.0
11 J0024-7204N
6.287107699025368e+18
12 J0024-7204O
5.367488077244541e+18
13 J0024-7204Q
9.595596311237536e+18
14 J0024-7204R
1.0804459731284754e+18
15 J0024-7204S
0.0
16 J0024-7204T
6.154004973358236e+17
17 J0024-7204U
2.1456733713280858e+18
18 J0024-7204Y
5.164304738620038e+18
19 J0024-7204Z
3.3262252990070364e+18


<ipython-input-5-8535a2bc3c70>:186: RuntimeWarning: divide by zero encountered in log
  logprobs = np.log(probs)


-inf